##### Copyright 2019 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2019 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Multilingual Universal Sentence Encoder Q&amp;A 검색


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/retrieval_with_tf_hub_universal_encoder_qa.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a>
  </td>
  <td><a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/retrieval_with_tf_hub_universal_encoder_qa.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 보기</a></td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/retrieval_with_tf_hub_universal_encoder_qa.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a>
  </td>
  <td data-parent-segment-id="12900598"><a href="https://tfhub.dev/s?q=google%2Funiversal-sentence-encoder-multilingual-qa%2F3%20OR%20google%2Funiversal-sentence-encoder-qa%2F3"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">TF Hub 모델 보기</a></td>
</table>

이 튜토리얼은 질문-대답 텍스트 검색에 [Univeral Encoder Multilingual Q&amp;A 모델](https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3)을 사용하여 모델의 **question_encoder** 및 **response_encoder** 사용을 예시하는 데모입니다. [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) 단락의 문장을 데모 데이터시트로 사용하고 각 문장과 해당 컨텍스트(문장을 둘러싼 텍스트)가 **response_encoder**를 통해 고차원 임베딩에 인코딩됩니다. 이러한 임베딩은 질문-대답 검색을 위해 <a>simpleneighbors</a> 라이브러리를 사용하여 빌드된 인덱스에 저장됩니다.

검색할 때 [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) 데이터세트에서 임의의 질문을 선택하고 **question_encoder**를 사용하여 고차원 임베딩으로 인코딩하며 의미론적 공간에서 ANN(approximate nearest neighbor) 목록을 반환하는 simpleneighbors 인덱스를 쿼리합니다.

### 더 많은 모델

[여기](https://tfhub.dev/s?module-type=text-embedding)에서 현재 호스팅되고 있는 모든 텍스트 내장 모델, 그리고 [여기](https://tfhub.dev/s?dataset=squad)에서 SQuAD에서 훈련된 모든 모델을 찾아볼 수 있습니다.

## Setup


In [ ]:
%%capture
#@title Setup Environment
# Install the latest Tensorflow version.
!pip install -q tensorflow_text
!pip install -q simpleneighbors[annoy]
!pip install -q nltk
!pip install -q tqdm

In [ ]:
#@title Setup common imports and functions
import json
import nltk
import os
import pprint
import random
import simpleneighbors
import urllib
from IPython.display import HTML, display
from tqdm.notebook import tqdm

import tensorflow.compat.v2 as tf
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer

nltk.download('punkt')


def download_squad(url):
  return json.load(urllib.request.urlopen(url))

def extract_sentences_from_squad_json(squad):
  all_sentences = []
  for data in squad['data']:
    for paragraph in data['paragraphs']:
      sentences = nltk.tokenize.sent_tokenize(paragraph['context'])
      all_sentences.extend(zip(sentences, [paragraph['context']] * len(sentences)))
  return list(set(all_sentences)) # remove duplicates

def extract_questions_from_squad_json(squad):
  questions = []
  for data in squad['data']:
    for paragraph in data['paragraphs']:
      for qas in paragraph['qas']:
        if qas['answers']:
          questions.append((qas['question'], qas['answers'][0]['text']))
  return list(set(questions))

def output_with_highlight(text, highlight):
  output = "<li> "
  i = text.find(highlight)
  while True:
    if i == -1:
      output += text
      break
    output += text[0:i]
    output += '<b>'+text[i:i+len(highlight)]+'</b>'
    text = text[i+len(highlight):]
    i = text.find(highlight)
  return output + "</li>\n"

def display_nearest_neighbors(query_text, answer_text=None):
  query_embedding = model.signatures['question_encoder'](tf.constant([query_text]))['outputs'][0]
  search_results = index.nearest(query_embedding, n=num_results)

  if answer_text:
    result_md = '''
    <p>Random Question from SQuAD:</p>
    <p>&nbsp;&nbsp;<b>%s</b></p>
    <p>Answer:</p>
    <p>&nbsp;&nbsp;<b>%s</b></p>
    ''' % (query_text , answer_text)
  else:
    result_md = '''
    <p>Question:</p>
    <p>&nbsp;&nbsp;<b>%s</b></p>
    ''' % query_text

  result_md += '''
    <p>Retrieved sentences :
    <ol>
  '''

  if answer_text:
    for s in search_results:
      result_md += output_with_highlight(s, answer_text)
  else:
    for s in search_results:
      result_md += '<li>' + s + '</li>\n'

  result_md += "</ol>"
  display(HTML(result_md))

다음 코드 블록을 실행하여 SQuAD 데이터세트를 다운로드하고 다음으로 추출합니다.

- **문장**은 (텍스트, 컨텍스트) 튜플의 목록입니다. SQuAD 데이터세트의 각 단락은 nltk 라이브러리를 사용하여 문장으로 분할되고 문장 및 단락 텍스트는 (텍스트, 컨텍스트) 튜플을 형성합니다.
- **질문**은 (질문, 답변) 튜플의 목록입니다.

참고: 이 데모를 사용하여 SQuAD 훈련 데이터세트 또는 아래 **squad_url**을 사용해 더 작은 개발 데이터세트 (1.1 또는 2.0)를 인덱싱할 수 있습니다.


In [ ]:
#@title Download and extract SQuAD data
squad_url = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json' #@param ["https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"]

squad_json = download_squad(squad_url)
sentences = extract_sentences_from_squad_json(squad_json)
questions = extract_questions_from_squad_json(squad_json)
print("%s sentences, %s questions extracted from SQuAD %s" % (len(sentences), len(questions), squad_url))

print("\nExample sentence and context:\n")
sentence = random.choice(sentences)
print("sentence:\n")
pprint.pprint(sentence[0])
print("\ncontext:\n")
pprint.pprint(sentence[1])
print()

다음 코드 블록은 [Univeral Encoder Multilingual Q&amp;A 모델](https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3)의 **question_encoder** 및 **response_encoder** 서명을 사용하여 tensorflow 그래프 **g** 및 **세션**을 설정합니다.

In [ ]:
#@title Load model from tensorflow hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3", "https://tfhub.dev/google/universal-sentence-encoder-qa/3"]
model = hub.load(module_url)


다음 코드 블록은 모든 텍스트, 컨텍스트 튜플에 대한 임베딩을 계산하고 **response_encoder**를 사용하여 [simpleneighbors](https://pypi.org/project/simpleneighbors/) 인덱스에 계산 값을 저장합니다.


In [ ]:
#@title Compute embeddings and build simpleneighbors index
batch_size = 100

encodings = model.signatures['response_encoder'](
  input=tf.constant([sentences[0][0]]),
  context=tf.constant([sentences[0][1]]))
index = simpleneighbors.SimpleNeighbors(
    len(encodings['outputs'][0]), metric='angular')

print('Computing embeddings for %s sentences' % len(sentences))
slices = zip(*(iter(sentences),) * batch_size)
num_batches = int(len(sentences) / batch_size)
for s in tqdm(slices, total=num_batches):
  response_batch = list([r for r, c in s])
  context_batch = list([c for r, c in s])
  encodings = model.signatures['response_encoder'](
    input=tf.constant(response_batch),
    context=tf.constant(context_batch)
  )
  for batch_index, batch in enumerate(response_batch):
    index.add_one(batch, encodings['outputs'][batch_index])

index.build()
print('simpleneighbors index for %s sentences built.' % len(sentences))


검색할 때 **question_encoder**를 사용하여 질문이 인코딩되고 질문 임베딩이 simpleneighbors 인덱스를 쿼리하는 데 사용됩니다.

In [ ]:
#@title Retrieve nearest neighbors for a random question from SQuAD
num_results = 25 #@param {type:"slider", min:5, max:40, step:1}

query = random.choice(questions)
display_nearest_neighbors(query[0], query[1])